In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from anthropic import AnthropicBedrock
from application import Application
from compiler.core import Compiler
from core.interpolator import Interpolator

In [3]:
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [4]:
import os
from dotenv import load_dotenv
env_path = "templates/app_schema/.env"
if os.path.exists(env_path):
    load_dotenv(env_path)
    print(f"Loaded environment variables from {env_path}")

Loaded environment variables from templates/app_schema/.env


In [5]:
application = Application(client, compiler)
interpolator = Interpolator(".")

In [ ]:
user_prompts = """Bot that says hello"""

In [7]:

my_bot = application.prepare_bot([user_prompts], bot_id="handler_tests", capabilities=["perplexity.web_search"])
my_bot.__dict__

Compiling TypeSpec...


{'refined_description': None,
 'capabilities': CapabilitiesOut(capabilities=['perplexity.web_search'], error_output=None),
 'typespec': TypespecOut(reasoning='Looking at the user request, they want a simple bot that says hello. The interface needs to handle greeting functionalities. I\'ll create a model and interface for this basic use case.\n\nKey functionalities needed:\n1. A way to greet the user - `greetUser()`\n2. Maybe a way to customize the greeting - passing a name parameter\n3. A way to respond to different greeting scenarios (standard, time-based, etc.)\n\nThe models will be simple:\n- `GreetingOptions` - containing optional name and locale\n- `TimeBasedGreetingOptions` - for time-sensitive greetings\n\nFor scenarios, I\'ll cover:\n- Basic greeting (just "hello")\n- Personalized greeting (with name)\n- Time-based greeting (good morning, good evening, etc.)', typespec_definitions='model GreetingOptions {\n  name?: string;\n  locale?: string;\n}\n\nmodel TimeBasedGreetingOption

In [13]:
my_bot = application.prepare_bot([user_prompts, "Add say goodbye"], bot_id="handler_tests", capabilities=["perplexity.web_search"])
my_bot.__dict__

Compiling TypeSpec...


{'refined_description': None,
 'capabilities': CapabilitiesOut(capabilities=['perplexity.web_search'], error_output=None),
 'typespec': TypespecOut(reasoning="From the user requests, I need to create a simple greeting bot interface with two functions:\n1. sayHello - Says hello to the user\n2. sayGoodbye - Says goodbye to the user\n\nEach function will need a simple options model to hold parameters, scenarios, and llm_func decorators.\n\nFor the sayHello function:\n- It needs an options parameter (as per the rules)\n- A scenario for a basic greeting\n- A scenario for a greeting with user's name\n\nFor the sayGoodbye function:\n- It needs an options parameter\n- A scenario for a basic farewell\n- A scenario for a farewell with user's name\n\nI'll create appropriate models for each function's options parameter. Since these are simple greeting functions, the options could include:\n- userName (optional): To personalize the greeting/farewell\n- useEmoji (optional): To include emoji in the g

In [ ]:
my_bot = application.update_bot(
    typespecSchema=my_bot.typespec.typespec_definitions, 
    bot_id="web_search_bot", 
    capabilities=["perplexity.web_search"]
)

Creating prompt from TypeSpec...
Processing TypeSpec...
Generating Typescript Schema...
Generating Drizzle Schema...
Generating Handler Tests...
Generating Handlers...


{'refined_description': None,
 'capabilities': CapabilitiesOut(capabilities=['perplexity.web_search'], error_output=None),
 'typespec': TypespecOut(reasoning="From the user requests, I need to create a simple greeting bot interface with two functions:\n1. sayHello - Says hello to the user\n2. sayGoodbye - Says goodbye to the user\n\nEach function will need a simple options model to hold parameters, scenarios, and llm_func decorators.\n\nFor the sayHello function:\n- It needs an options parameter (as per the rules)\n- A scenario for a basic greeting\n- A scenario for a greeting with user's name\n\nFor the sayGoodbye function:\n- It needs an options parameter\n- A scenario for a basic farewell\n- A scenario for a farewell with user's name\n\nI'll create appropriate models for each function's options parameter. Since these are simple greeting functions, the options could include:\n- userName (optional): To personalize the greeting/farewell\n- useEmoji (optional): To include emoji in the g

AttributeError: 'NoneType' object has no attribute 'drizzle_schema'

In [ ]:
my_bot.__dict__

{'refined_description': RefineOut(refined_description='model HelloOptions {\n  name: string;\n  style?: string;\n}\n\nmodel GoodbyeOptions {\n  name: string;\n  style?: string;\n}\n\ninterface GreetingBot {\n  @scenario("""\n  Scenario: Saying hello to a person\n  When user says "Say hello to John"\n  Then system greets John\n  \n  Examples:\n    | Input                     | Expected Output          |\n    | "Hello to Mary"           | "Hello, Mary!"           |\n    | "Say hi to Bob formally"  | "Good day, Bob."         |\n  """)\n  @llm_func("Generate a greeting for a person")\n  sayHello(options: HelloOptions): void;\n\n  @scenario("""\n  Scenario: Saying goodbye to a person\n  When user says "Say goodbye to Sarah"\n  Then system says goodbye to Sarah\n  \n  Examples:\n    | Input                       | Expected Output           |\n    | "Goodbye to Tom"            | "Goodbye, Tom!"           |\n    | "Say bye to Lisa formally"  | "Farewell, Lisa."         |\n  """)\n  @llm_func("

In [ ]:
interpolator.bake(my_bot, "../../bot_result") # next to repo root